In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:

prerequisitos_raw = pd.read_csv('../dados/prerequisitos_raw.csv', delimiter=';')
disciplinas = pd.read_csv('../dados/disciplinas.csv', delimiter=';')
matriculas =  pd.read_csv('../dados/matriculas.csv', delimiter=';')
alunos = pd.read_csv('../dados/alunos.csv', delimiter=';')


In [3]:

alunos_ativos = alunos.loc[
        (alunos['ALU_FORMA_EVASAO'] == 0) &
        (alunos['PERIODO_INGRESSAO'] <= 2017.1)
    ]

mat_alunos_atvs = list(alunos_ativos['ALU_MATRICULA'])


In [4]:

historico_mat_alunos_atv = matriculas.loc[
        (matriculas['MAT_ALU_MATRICULA'].isin(mat_alunos_atvs)) &
        (matriculas['PERIODO_MAT'] <= 2017.1)
]


In [5]:

disciplinas_filtered = disciplinas.loc[
    (   (disciplinas['DIC_REGRA'] == 'Optativa') | (disciplinas['DIC_REGRA'] == 'Complementar') |
        ((disciplinas['DIC_REGRA'] == 'Obrigatoria') & (disciplinas['DIC_STATUS'] == 'A'))
    ) & (disciplinas['DIS_DISCIPLINA'].isin(historico_mat_alunos_atv['MAT_TUR_DIS_DISCIPLINA']))    
]

set_cod_disciplinas = list(set( disciplinas_filtered['DIS_DISCIPLINA'] ))


In [6]:

disciplina_prerequisitos = dict()
col_names = dict((col,index) for index, col in enumerate(list(prerequisitos_raw.columns.values)))

for disc, obs in prerequisitos_raw.groupby(['PRD_DIC_DIS_DISCIPLINA']):

    discis_ou, discis_e, curr_ou = [], [], []
    for row in obs.get_values():
                        
        if row[col_names['PRD_ORDEM_PRIOR']] == 1:
            if row[col_names['PRD_OPERADOR']] == 'E ' or str(row[col_names['PRD_OPERADOR']]) == 'nan':
                discis_e.append(row[col_names['PRD_CONDICAO']])
            
            elif row[col_names['PRD_OPERADOR']] == 'OU':
                curr_ou.append(row[col_names['PRD_CONDICAO']])
            
            ult_operador = row[col_names['PRD_OPERADOR']]
            
        else:
            if ult_operador == 'OU':
                curr_ou.append(row[col_names['PRD_CONDICAO']])
            
            elif ult_operador == 'E ':
                # Entao um conjunto de OUs acabou
                discis_ou.append(curr_ou)
                curr_ou = []
                
                # Entao eh uma disciplina sozinha que eh prerequisito
                if row[col_names['PRD_OPERADOR']] == 'E ':
                    discis_e.append(row[col_names['PRD_CONDICAO']])
                    
                # Entao começa um conjunto de OUs em que pelo menos um eh prerequisito
                #   ou é o último prerequisito
                else:
                    if str(row[col_names['PRD_OPERADOR']]) == 'nan':
                        discis_e.append(row[col_names['PRD_CONDICAO']])
                    
                    else:
                        curr_ou.append(row[col_names['PRD_CONDICAO']])
                
            ult_operador = row[col_names['PRD_OPERADOR']]
            
    discis_ou.append(curr_ou)
    meus_prerequisitos = {'OU': discis_ou, 'E': discis_e}
    disciplina_prerequisitos[disc] = meus_prerequisitos
      
# seria bom depois ver se algum prerequisito de uma disciplina ativa é uma disciplina inativa


In [7]:

aluno_historico = dict()
col_names = dict(
    (col, index) for index, col in enumerate(
            list(historico_mat_alunos_atv.columns.values)
    )
)

for aluno, obs in historico_mat_alunos_atv.groupby(['MAT_ALU_MATRICULA']):
    
    meu_historico = []
    for row in obs.get_values():
        
        # Aluno foi aprovado na disciplina
        if row[col_names['MAT_SITUACAO']] == 3:
            meu_historico.append( row[col_names['MAT_TUR_DIS_DISCIPLINA']] )
    
    aluno_historico[aluno] = meu_historico
    

In [8]:

to_data_frame = []

for cod_disc in set_cod_disciplinas:
    
    # Se disciplina não tem pré-requisito
    if cod_disc not in disciplina_prerequisitos.keys():
        continue
        
    count_alunos = 0
    disc_prerequisito = disciplina_prerequisitos[cod_disc]
    nome_disciplina = disciplinas.loc[(disciplinas['DIS_DISCIPLINA'] == cod_disc)]['DIS_DESCRICAO'].values[0]
    
    for aluno in mat_alunos_atvs:
        
        aluno_hist = aluno_historico[aluno]
        
        # Se aluno já pagou cadeira, não faça nada
        if cod_disc in aluno_hist:
            continue
                
        # found -> cumpriu todos os pré-requisitos
        found = True
        
        # Se para cada conjunto de disciplinas em que ter pago pelo menos uma é pré-requisito
        for ou_list in disc_prerequisito['OU']:
            if len(ou_list) > 0:
                found = found and any(disciplina_OU in aluno_hist for disciplina_OU in ou_list)
        
        # Se pagou todas as disciplinas
        found = found and all(disciplina_E in aluno_hist for disciplina_E in disc_prerequisito['E'])
        
        count_alunos = count_alunos + 1 if found else count_alunos
        
    nome_disciplina = disciplinas.loc[(disciplinas['DIS_DISCIPLINA'] == cod_disc)]['DIS_DESCRICAO'].values[0]
    new_row = {'COD_DISCIPLINA': cod_disc, 
               'NOME_DISCIPLINA': nome_disciplina,
               'QNT_ALUNOS_APTOS': count_alunos}
    to_data_frame.append(new_row)
    

In [12]:

df = pd.DataFrame(columns=('COD_DISCIPLINA', 'NOME_DISCIPLINA', 'QNT_ALUNOS_APTOS'))
df = pd.DataFrame(to_data_frame)
df.to_csv('../dados/disciplinas_qnt_alunos_aptos.csv', index=False)
